In [9]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

import openmm as mm
import openmm.app as app
import openmm.unit as unit
from mdtraj.reporters import HDF5Reporter
import copy
import nglview as nv
import random

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def show_pdb(traj):
    view = nv.show_mdtraj(traj)
    view.clear()
    chain_a  = traj.top.select('chainid 0')
    chain_b  = traj.top.select('chainid 1')
    view.add_representation('licorice', selection=chain_a, color='cornflowerblue')
    view.add_representation('licorice', selection=chain_b, color='coral')
    return view

In [35]:
# Make DNA from sequence
sequence = 'CATCATCAT'
pdb = mdna.sequence_to_pdb(sequence=sequence,output='GROMACS')
pdb.save('./pdbs/wild.pdb')
show_pdb(pdb)

Sequence: CATCATCAT
Number of base pairs: 9


NGLWidget()

In [36]:
# Mutate DNA 
mutations = {0: 'A', 1: 'T', 2: 'G'}
mutant = mdna.Mutate(pdb, mutations,complementary=True)
mutant_pdb = mutant.get_traj()
mutant_pdb.save('./pdbs/mutant.pdb')

mutant_sequence = mdna.get_sequence_letters(mutant_pdb)
print('New sequence:',''.join(mutant_sequence))
view = show_pdb(mutant_pdb)
selection = mutant_pdb.top.select('resid 0 1 2 17 16 15')
view.add_representation('licorice', selection=selection, color='green')
view

New sequence: ATGCATCAT


NGLWidget()

In [37]:
# Mutation (non-canonical//fluorescent base analogue)
# P = 2AP (2-aminopurine)  https://doi.org/10.1002/anie.201001312 (2kv0)
# D = tricyclic cytosin base analogue (1tuq)
# Hydrophobic bases! Hachimoji DNA DOI: 10.1126/science.aat0971
mutations = {1: 'P', 2 : 'D'}
mutant = mdna.Mutate(pdb, mutations,complementary=True)
new_pdb = mutant.get_traj()
new_pdb.save('./pdbs/baseanalogues.pdb')

mutant_sequence = mdna.get_sequence_letters(new_pdb)
print('New sequence:',''.join(mutant_sequence))
view = show_pdb(new_pdb)
selection = new_pdb.top.select('resid 1 2')
view.add_representation('licorice', selection=selection, color='yellow')
view

New sequence: CPDCATCAT


NGLWidget()

In [38]:
# Methylate DNA
methylator = mdna.Methylate(pdb, methylations=[0])
pdb_meth = methylator.get_traj()
pdb_meth.save('./pdbs/methylated.pdb')

meth_sequence = mdna.get_sequence_letters(pdb_meth)
print('New sequence:',''.join(meth_sequence))
view = show_pdb(pdb_meth)
selection = pdb_meth.top.select('resid 0')
view.add_representation('licorice', selection=selection, color='purple')
view

New sequence: CATCATCAT


NGLWidget()

In [39]:
# Methylate DNA
sequence = 'GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG'
long_pdb = mdna.sequence_to_pdb(sequence=sequence,output='GROMACS')
methylator = mdna.Methylate(long_pdb, CpG=True,leading_strand=0)
pdb_meth_long = methylator.get_traj()

pdb_meth_long.save('./pdbs/methylated.pdb')
meth_sequence = mdna.get_sequence_letters(pdb_meth_long)
print('New sequence:',''.join(meth_sequence))
view = show_pdb(pdb_meth_long)
resids = methylator.baselist
selection = pdb_meth_long.top.select('resid '+' '.join([str(i) for i in resids]))
view.add_representation('licorice', selection=selection, color='purple')
view

Sequence: GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG
Number of base pairs: 43
Methylate all C in CpG context, superseeds methylations list.
Methtylating: [1, 8, 10, 13, 15, 19, 25, 31, 41]
New sequence: GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG


NGLWidget()

In [40]:
# Hoogsteen flip
flipper = mdna.Hoogsteen(pdb,fliplist=[1],deg=180,verbose=True)
pdb_flipped = flipper.get_traj()
pdb_flipped.save('./pdbs/WC_to_HG.pdb')
view = show_pdb(pdb_flipped)
selection = pdb_meth_long.top.select('resid '+' '.join([str(i) for i in flipper.fliplist]))
view.add_representation('licorice', selection=selection, color='cyan')
view

Flipped residues [1] by 3.141592653589793 radians


NGLWidget()

In [41]:
def point_mutation(sequence, position=None, new_nucleotide=None):
    if position is None:
        position = random.randint(0, len(sequence) - 1)
    if new_nucleotide is None:
        nucleotides = ['A', 'T', 'C', 'G']
        new_nucleotide = random.choice([n for n in nucleotides if n != sequence[position]])
    
    mutated_sequence = list(sequence)
    mutated_sequence[position] = new_nucleotide
    return ''.join(mutated_sequence)

def radiate_system(traj, new_sequence, complementary=True, chainids=[0,1], verbose=False):

    pdb = copy.deepcopy(traj)   
    if verbose:
        print('--- current stat of the system ---')
    s = mdna.get_sequence_letters(pdb)
    if len(s) != len(new_sequence):
        raise ValueError('The length of the new sequence does not match the length of the current sequence')
    if verbose:
        print(len(s),s)
        print(''.join(s))
    mutations = mdna.get_mutations(s,new_sequence)
    if verbose:
        print(f'start mutation ---- {mutations} ----')    
    dna = pdb.atom_slice(pdb.top.select(f'chainid {chainids[0]} {chainids[1]}'))
    if verbose:
        for c in dna.top.chains:
            print(c.index, c._residues)

    mutant = mdna.Mutate(dna, mutations,complementary=complementary)
    mutant_sequence = mdna.get_sequence_letters(mutant.mutant_traj)
    if verbose:
        print(mutant_sequence)
        print(''.join(mutant_sequence))
    new_traj = mutant.get_traj()
    if verbose:
        for c in new_traj.top.chains:
            print(c.index, c._residues)
        print('--- end radiation ---')
    return new_traj

# Example usage
sequence = mdna.get_sequence_letters(pdb)
point_mutations = np.unique([point_mutation(sequence) for _ in range(100)])

for i, new_sequence in enumerate(point_mutations):
    mutant = radiate_system(pdb, list(new_sequence))
    print(new_sequence,f'saved as point_mutant_{i}.pdb')
    mutant.save(f'./pdbs/point_mutant_{i}.pdb')


AATCATCAT saved as point_mutant_0.pdb
CAACATCAT saved as point_mutant_1.pdb
CACCATCAT saved as point_mutant_2.pdb
CAGCATCAT saved as point_mutant_3.pdb
CATAATCAT saved as point_mutant_4.pdb
CATCAACAT saved as point_mutant_5.pdb
CATCACCAT saved as point_mutant_6.pdb
CATCAGCAT saved as point_mutant_7.pdb
CATCATAAT saved as point_mutant_8.pdb
CATCATCAA saved as point_mutant_9.pdb
CATCATCAC saved as point_mutant_10.pdb
CATCATCAG saved as point_mutant_11.pdb
CATCATCCT saved as point_mutant_12.pdb
CATCATCGT saved as point_mutant_13.pdb
CATCATCTT saved as point_mutant_14.pdb
CATCATGAT saved as point_mutant_15.pdb
CATCATTAT saved as point_mutant_16.pdb
CATCCTCAT saved as point_mutant_17.pdb
CATCGTCAT saved as point_mutant_18.pdb
CATCTTCAT saved as point_mutant_19.pdb
CATGATCAT saved as point_mutant_20.pdb
CATTATCAT saved as point_mutant_21.pdb
CCTCATCAT saved as point_mutant_22.pdb
CGTCATCAT saved as point_mutant_23.pdb
CTTCATCAT saved as point_mutant_24.pdb
GATCATCAT saved as point_mutant_25.